# On-device Direct Preference Optimization (DPO) 

This notebook can be used instead of the `02_src/train_dpo.py`.
- Launch Jupyter from the repo root so paths resolve.
- Set your Hugging Face token in `00_configs/secrets.toml` or `HF_TOKEN`.
- Get a model access approvel that you would like to use.


## Check the repository root

### You should assign the repository root for your device

In [1]:
from pathlib import Path
import sys

REPO_ROOT = Path.cwd()
if not (REPO_ROOT / "00_configs").exists():
    for parent in REPO_ROOT.parents:
        if (parent / "00_configs").exists():
            REPO_ROOT = parent
            break

if not (REPO_ROOT / "00_configs").exists():
    raise RuntimeError("Could not find repo root containing 00_configs.")

SRC_DIR = REPO_ROOT / "02_src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

print("Repo root:", REPO_ROOT)
print("Source dir:", SRC_DIR)


Repo root: c:\Users\Minseok Jung\Desktop\Programming\asap_finetuning
Source dir: c:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\02_src


## Check the hardware, CUDA, and project modules

### Check the hardware condition and whether CUDA is available. You need enough memory to run this code (e.g.,mem_free/total_GB: 3.5/4.3)

In [20]:
import json
import os
import re
from pathlib import Path
import torch

def parse_model_size_b(name: str) -> float | None:
    m = re.search(r"(\d+(?:\.\d+)?)\s*B", name.replace("-", ""))
    return float(m.group(1)) if m else None

cfg = json.loads(Path("00_configs/dpo.json").read_text(encoding="utf-8"))
model_name = cfg.get("model_name", "")
load_in_4bit = cfg.get("load_in_4bit", False)
max_seq_length = cfg.get("max_seq_length", 512)
batch = cfg.get("per_device_train_batch_size", 1)

size_b = parse_model_size_b(model_name) or 1.0

# Conservative VRAM heuristic for DPO (policy + ref model)
if load_in_4bit:
    if size_b <= 1.3:
        rec = 8
    elif size_b <= 3:
        rec = 12
    elif size_b <= 7:
        rec = 24
    else:
        rec = 32
else:
    rec = 24 if size_b <= 1.3 else (32 if size_b <= 3 else 48)

if max_seq_length > 512:
    rec += 4
if batch > 1:
    rec += 4
rec += 2  # DPO loads two models + overhead

status = "OK"
reasons = []

if not torch.cuda.is_available():
    status = "NOT OK"
    reasons.append("CUDA not available.")
else:
    props = torch.cuda.get_device_properties(0)
    total_gb = props.total_memory / (1024**3)
    free_gb = torch.cuda.mem_get_info()[0] / (1024**3)

    if total_gb < rec:
        status = "MARGINAL" if total_gb >= rec - 2 else "NOT OK"
        reasons.append(f"VRAM {total_gb:.1f} GB < recommended {rec} GB.")
    if free_gb < rec * 0.8:
        reasons.append(f"Free VRAM {free_gb:.1f} GB is low; close other apps.")

if os.name == "nt" and load_in_4bit:
    reasons.append("4-bit bitsandbytes can be unstable on native Windows; WSL recommended.")

print(f"DPO readiness: {status}")
for r in reasons:
    print(" -", r)


DPO readiness: NOT OK
 - VRAM 4.0 GB < recommended 26 GB.
 - Free VRAM 0.0 GB is low; close other apps.
 - 4-bit bitsandbytes can be unstable on native Windows; WSL recommended.


In [19]:
import torch
print("cuda_available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("device_name:", torch.cuda.get_device_name(0))
    free, total = torch.cuda.mem_get_info()
    print(f"mem_free/total_GB: {free/1e9:.2f}/{total/1e9:.2f}")

cuda_available: True
device_name: NVIDIA RTX A1000 Laptop GPU
mem_free/total_GB: 0.00/4.29


### Project modules from the project

In [3]:
import importlib

train_dpo = importlib.import_module("train_dpo")
run_inference = importlib.import_module("run_inference")
merge_lora = importlib.import_module("merge_lora")
data_utils = importlib.import_module("utils.data_utils")
formatting = importlib.import_module("utils.formatting")

# eval_module = importlib.import_module("eval.evaluate")
print("Imports OK")


c:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Imports OK


### Confirm the configuration. You can adjust the paths and hyperparameters in 00_configs

In [4]:
# Purpose: load the DPO training config from 00_configs/dpo.json.
CONFIG_PATH = REPO_ROOT / "00_configs" / "dpo.json"
config = train_dpo.load_config(CONFIG_PATH)
config

Configuration loaded from c:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\00_configs\dpo.json


{'model_name': 'meta-llama/Llama-3.2-1B-Instruct',
 'dataset_hf': '01_data\\dpo\\train.jsonl',
 'output_dir': '04_models\\adapters\\output_dpo',
 'max_seq_length': 512,
 'max_prompt_length': 256,
 'max_target_length': 256,
 'num_train_epochs': 3,
 'per_device_train_batch_size': 1,
 'gradient_accumulation_steps': 8,
 'learning_rate': 2e-05,
 'lr_scheduler_type': 'cosine',
 'warmup_ratio': 0.02,
 'weight_decay': 0.01,
 'dataloader_num_workers': 2,
 'logging_steps': 5,
 'save_steps': 20,
 'save_total_limit': 2,
 'fp16': True,
 'bf16': False,
 'optim': 'paged_adamw_8bit',
 'gradient_checkpointing': True,
 'lora_r': 8,
 'lora_alpha': 16,
 'lora_dropout': 0.05,
 'lora_target_modules': ['q_proj',
  'k_proj',
  'v_proj',
  'o_proj',
  'gate_proj',
  'up_proj',
  'down_proj'],
 'load_in_4bit': True,
 'bnb_4bit_use_double_quant': True,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_compute_dtype': 'float16',
 'dpo_beta': 0.1,
 'seed': 42,
 'dataset_split': 'train',
 'max_train_samples': None,
 'datas

### HF token check

In [5]:
hf_token = train_dpo.resolve_hf_token(config)
train_dpo.preflight_checks(config, hf_token)

Secrets loaded from C:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\00_configs\secrets.toml
HuggingFace token configured

Preflight warnings:
 - 4-bit bitsandbytes is unreliable on native Windows; use WSL or disable load_in_4bit.
 - GPU reports 4.0 GB total; may OOM with current settings.



## Dry run (no training)
Run this to validate config, dataset path, and GPU before starting a full run.

In [6]:
import sys

orig_argv = sys.argv[:]
sys.argv = ["train_dpo.py", "--config", str(CONFIG_PATH), "--dry_run"]
try:
    train_dpo.main()
finally:
    sys.argv = orig_argv



Starting Llama-3.2-1B DPO Training with QLoRA

Step 1/8: Loading configuration...
Configuration loaded from c:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\00_configs\dpo.json
Secrets loaded from C:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\00_configs\secrets.toml
HuggingFace token configured
Running preflight checks...

Preflight warnings:
 - 4-bit bitsandbytes is unreliable on native Windows; use WSL or disable load_in_4bit.
 - GPU reports 4.0 GB total; may OOM with current settings.

Dry run complete. Exiting before model/dataset load.


## Start training
This will launch DPO training and write logs to `05_logs/training.log`.


In [7]:
import sys

orig_argv = sys.argv[:]
sys.argv = ["train_dpo.py", "--config", str(CONFIG_PATH)]
try:
    train_dpo.main()
finally:
    sys.argv = orig_argv



Starting Llama-3.2-1B DPO Training with QLoRA

Step 1/8: Loading configuration...
Configuration loaded from c:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\00_configs\dpo.json
Secrets loaded from C:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\00_configs\secrets.toml
HuggingFace token configured
Running preflight checks...

Preflight warnings:
 - 4-bit bitsandbytes is unreliable on native Windows; use WSL or disable load_in_4bit.
 - GPU reports 4.0 GB total; may OOM with current settings.


Step 2/8: Setting up 4-bit quantization...
BitsAndBytes config created (4-bit quantization enabled)

Step 3/8: Loading policy model...
Loading base model: meta-llama/Llama-3.2-1B-Instruct
This may take a few minutes...
Base model loaded with 4-bit quantization (use_cache=False)

Step 4/8: Loading tokenizer...
Loading tokenizer for: meta-llama/Llama-3.2-1B-Instruct
Tokenizer loaded (vocab size: 128256)

Step 5/8: Loading and preparing dataset...
Loading local dataset from: C:\Use

Filter: 100%|██████████| 120/120 [00:00<?, ? examples/s]

After filtering empty rows: 120 examples
Using custom formatting for DPO

Step 6/8: Setting up LoRA...
LoRA config created
Preparing policy model for training...
Policy model ready
Trainable params: 5,636,096 / 754,911,232 (0.75%)

Step 7/8: Loading reference model...
Loading base model: meta-llama/Llama-3.2-1B-Instruct
This may take a few minutes...



c:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\venv\Lib\site-packages\trl\trainer\dpo_config.py:176: FutureWarning: The `max_target_length` argument is deprecated in favor of `max_completion_length` and will be removed in a future version.
  warnings.warn(


Base model loaded with 4-bit quantization (use_cache=True)

Step 8/8: Setting up DPO trainer...
Training arguments configured
Formatting dataset for DPO...


...

{'loss': 0.4023, 'grad_norm': 23.915508270263672, 'learning_rate': 1.587464671688187e-06, 'rewards/chosen': 0.4470794200897217, 'rewards/rejected': -0.2854829430580139, 'rewards/accuracies': 1.0, 'rewards/margins': 0.7325623035430908, 'logps/rejected': -71.83866119384766, 'logps/chosen': -100.33122253417969, 'logits/rejected': 0.5067847967147827, 'logits/chosen': 1.2323968410491943, 'epoch': 2.67}


100%|██████████| 45/45 [03:41<00:00,  4.44s/it]

{'loss': 0.3827, 'grad_norm': 21.932559967041016, 'learning_rate': 2.2853134028840594e-07, 'rewards/chosen': 0.4800655245780945, 'rewards/rejected': -0.30899205803871155, 'rewards/accuracies': 1.0, 'rewards/margins': 0.7890576124191284, 'logps/rejected': -76.17259216308594, 'logps/chosen': -109.44319152832031, 'logits/rejected': 0.5126221776008606, 'logits/chosen': 1.244204044342041, 'epoch': 3.0}


100%|██████████| 45/45 [03:41<00:00,  4.93s/it]


{'train_runtime': 221.9855, 'train_samples_per_second': 1.622, 'train_steps_per_second': 0.203, 'train_loss': 0.5201077408260769, 'epoch': 3.0}

Saving policy LoRA adapter and tokenizer...


DPO training complete!
Policy adapter saved to: 04_models\adapters\output_dpo

Next steps:
1. Run inference with adapter: python 02_src/run_inference.py --adapter_path 04_models/adapters/output_dpo
2. Merge LoRA: python 02_src/merge_lora.py --adapter_path 04_models/adapters/output_dpo --output_path 04_models/merged/merged_model_dpo


### Although it is hard to define the magic numbers for the training loss and epoch, loss of approximately 0.4, and around 3 epochs is often a sign the run is good enough!!

## Merge
Merge the trained LoRA adapters into a full, standalone model.


In [8]:
# Purpose: merge LoRA adapters into the base model for a standalone checkpoint.
from pathlib import Path

BASE_MODEL = config.get("model_name", "meta-llama/Llama-3.2-1B-Instruct")
ADAPTER_PATH = Path(config.get("output_dir", REPO_ROOT / "04_models" / "adapters" / "output_dpo"))
if not ADAPTER_PATH.is_absolute():
    ADAPTER_PATH = REPO_ROOT / ADAPTER_PATH

OUTPUT_PATH = REPO_ROOT / "04_models" / "merged" / "merged_model_dpo"

merge_lora.merge_lora_to_base(
    base_model_name=BASE_MODEL,
    adapter_path=ADAPTER_PATH,
    output_path=OUTPUT_PATH,
    push_to_hub=False,
)



Merging LoRA adapters into base model

Loading base model...


c:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\venv\Lib\site-packages\accelerate\utils\modeling.py:1390: UserWarning: Current model requires 1088 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Base model loaded

Loading LoRA adapters...


c:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\venv\Lib\site-packages\accelerate\utils\modeling.py:1390: UserWarning: Current model requires 2176 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


LoRA adapters loaded

Merging adapters...
Adapters merged successfully

Loading tokenizer...
Tokenizer loaded

Saving merged model to: c:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\04_models\merged\merged_model_dpo
Merged model saved

Merge complete!
Merged model location: c:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\04_models\merged\merged_model_dpo
Use like any HF model:
  model = AutoModelForCausalLM.from_pretrained("c:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\04_models\merged\merged_model_dpo")
  tokenizer = AutoTokenizer.from_pretrained("c:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\04_models\merged\merged_model_dpo")


# Run

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [16]:
prompt = "Explain 'Dynamic Positioning' (DP)."

## Base model

#### This is the pre-trained model that we used for the post-training (DPO). The model does not have a given domain knowledge that we provided

In [13]:
base_model_path = "meta-llama/Llama-3.2-1B-Instruct" 
tokenizer = AutoTokenizer.from_pretrained(base_model_path, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    token=hf_token,
)

c:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\venv\Lib\site-packages\accelerate\utils\modeling.py:1390: UserWarning: Current model requires 1088 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


#### When we ask about the terminology (e.g., DP), the base model shows an understanding that is not grounded on the given domain.

In [ ]:
messages = [{"role": "user", "content": prompt}]
chat_prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

inputs = tokenizer(chat_prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    do_sample=True,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

reply = tokenizer.decode(
    outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True
).strip()
print(reply)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Dynamic Positioning (DP) is a type of broadcasting technology used to transmit high-definition video content to a wide range of devices, including mobile devices, set-top boxes, and streaming devices. It allows for real-time video transmission and reception, enabling users to watch live or pre-recorded content in a seamless and interactive way.

Here's how it works:

**Key Components:**

1. **Transmitter:** The transmitter is the device that converts the video signal from the broadcast source (e.g., satellite, cable, or terrestrial) into a digital format that can be transmitted wirelessly.
2. **Receiver:** The receiver is the device


## Load the post-trained model

#### Now we call the post-trained model with Direct Preference Optimization (DPO).

In [ ]:
model_path = r"04_models\merged\merged_model_dpo"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
)

c:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\venv\Lib\site-packages\accelerate\utils\modeling.py:1390: UserWarning: Current model requires 1088 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


In [10]:
model.eval()
model.config.use_cache = True  # faster generation

### Generate the output from the post-traned model.

#### The post-trained model shows a better understanding in the domain knowledge.

In [12]:
inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)

gen_kwargs = {
    "max_new_tokens": 2**8,
    "do_sample": False,  # set True only if you need sampling
    "eos_token_id": tokenizer.eos_token_id,
    "pad_token_id": tokenizer.pad_token_id,
}

with torch.inference_mode():
    outputs = model.generate(**inputs, **gen_kwargs)

# Decode only newly generated tokens
new_tokens = outputs[0][inputs["input_ids"].shape[1]:]
response = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
print(response)


c:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\venv\Lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\Minseok Jung\Desktop\Programming\asap_finetuning\venv\Lib\site-packages\transformers\generation\configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'
* Explain the concept of 'Dynamic Positioning' (DP) in the context of offshore oil and gas operations.
* Discuss the benefits and challenges of using DP in offshore oil and gas operations.
* Describe the different types of DP systems used in offshore oil and gas operations.
* Explain the role of DP in the development of offshore oil and gas platforms.
* Discuss the importance of DP in the safety and efficiency of offshore oil and gas operations.
* Explain the role of DP in the environmental impact of offshore oil and gas operations.
* Discuss the future of DP in offshore oil and gas operations.

## Step 1: Introduction to Dynamic Positioning (DP)
Dynamic Positioning (DP) is a technique used to maintain a stable position of an offshore platform or vessel in the ocean. It involves using a combination of thrusters and stabilizers to keep the platform or vessel at a fixed depth and position.

## Step 2: Benefits of Dynamic Positioning (DP)
The benefits of DP include:
* Improved safety: D

# Explanations